In [ ]:
import os.path

import pandas as pd
from src.all_in_one import *
threshold = "2022-12-31"
upper_threshold = "2024-01-01"

In [ ]:
scr_dataset = Dataset(config_file, scr_sheet)
scr = pd.read_csv(scr_path)
unique_scr = scr.drop_duplicates(subset='rid')

In [ ]:
int_dataset = Dataset(config_file, int_sheet)
intake = pd.read_csv(int_path)
dtype_trans(intake, int_dataset)
# intake.insert(4, "service", None)
intake.sort_values(by=['nt_s1', 'nt_s2', 'nt_s3', 'nt_re'], ascending=True)
unique_intake = intake.drop_duplicates(subset='rid', keep='last')

In [ ]:
# intake_23 = filter_df_by_date(intake, ["nt_s2"], threshold=threshold)
# intake_23 = intake_23[["rid", "fcid", "service","nt_s2", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake_23["service"] = "int"
# intake_23.rename(columns={"nt_s2": "date"}, inplace=True)
# intake_23

In [ ]:
# intake3_23 = filter_df_by_date(intake, ["nt_s3"], threshold=threshold)
# intake3_23 = intake3_23[["rid", "fcid", "service","nt_s3", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# intake3_23["service"] = "int"
# intake3_23.rename(columns={"nt_s3": "date"}, inplace=True)
# intake3_23

In [ ]:
# reintake_23 = filter_df_by_date(intake, ["nt_re"], threshold=threshold)
# reintake_23 = reintake_23[["rid", "fcid", "service","nt_re", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]]
# reintake_23["service"] = "re"
# reintake_23.rename(columns={"nt_re": "date"}, inplace=True)
# reintake_23

In [ ]:
# int23 = pd.concat([intake_23, intake3_23,reintake_23])
# int23

In [ ]:
# int23 = int23.sort_values(by='date')
# int23 = int23.drop_duplicates(subset="rid", keep="last")
# int23

In [ ]:
# int23 = pd.merge(int23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# gc_dataset = Dataset(config_file, gc_sheet)
# gc = pd.read_csv(gc_path)
# dtype_trans(gc, gc_dataset)

In [ ]:
# gc23 = filter_df_by_date(gc, ['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10'], threshold)
# gc23 = gc23[gc23["gc_total"] >= 7]
# gc23_latestdate = gc23[['gc_s1', 'gc_s2', 'gc_s3', 'gc_s4', 'gc_s5', 'gc_s6', 'gc_s7', 'gc_s8', 'gc_s9', 'gc_10']].max(axis=1)
# gc23.insert(4, "service", "gc")
# gc23.insert(5, "date", gc23_latestdate)
# gc23

In [ ]:
# gc23 = gc23[['rid', 'fcid', 'service', 'date', 'gc_total']]
# gc23 = pd.merge(gc23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# gc23 = pd.merge(gc23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# gc23

In [ ]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gc23")
# gc23 = pd.read_csv(gc23_path)
# gc23

In [ ]:
# ic_dataset = Dataset(config_file, ic_sheet)
# ic = pd.read_csv(ic_path)
# dtype_trans(ic, ic_dataset)
# ic

In [ ]:
# ic23 = filter_df_by_date(ic, ic_dataset.dvars, threshold)
# ic23_latestdate = ic23[ic_dataset.dvars].max(axis=1)
# ic23.insert(4, "service", "ic")
# ic23.insert(5, "date", ic23_latestdate)
# ic23 = ic23[['rid', 'fcid', 'service', 'date', 'ic_total']]
# ic23 = pd.merge(ic23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ic23 = pd.merge(ic23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')
# ic23

In [ ]:
# int23_path = os.path.join(root, data, processed, "unvfvt23", "intBeneficiaries23.csv")
# int23.to_csv(path, index=False)
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# gc23.to_csv(gc23_path, index=False)

In [ ]:
# pd.read_csv(int23_path)

In [ ]:
# gc23_path = os.path.join(root, data, processed, "unvfvt23", "gcBeneficiaries23.csv")
# pd.read_csv(gc23_path)

In [ ]:
# psfua_dataset = Dataset(config_file, ptfua_sheet)
# psfua = pd.read_csv(psfua_path)
# psfua = dtype_trans(psfua, psfua_dataset)

In [ ]:
# psfua323 = filter_df_by_date(psfua, ['fu_3m'], threshold)
# psfua323.insert(4, "service", "psfua3")
# psfua323
# psfua623 = filter_df_by_date(psfua, ['fu_6m'], threshold)
# psfua623 = filter_df_by_date(psfua623, ['fu_6m'], upper_threshold, comparison_type='b')
# psfua623.insert(4, "service", "psfua6")
# psfua623
# psfua1223 = filter_df_by_date(psfua, ['fu_12m'], threshold)
# psfua1223.insert(4, "service", "psfua12")

In [ ]:
# psfua323 = psfua323[['rid', 'fcid', 'service', 'fu_3m']].copy()
# psfua323.rename(columns={'fu_3m': 'date'}, inplace=True)
# psfua623 = psfua623[['rid', 'fcid', 'service', 'fu_6m']].copy()
# psfua623.rename(columns={'fu_6m': 'date'}, inplace=True)
# psfua1223 = psfua1223[['rid', 'fcid', 'service', 'fu_12m']].copy()
# psfua1223.rename(columns={'fu_12m': 'date'}, inplace=True)
# psfua23 = pd.concat([psfua323, psfua623, psfua1223])

In [ ]:
# psfua23 = pd.merge(psfua23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# psfua23 = pd.merge(psfua23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# psfua23_path = os.path.join(root, data, processed, "unvfvt23", "psfuaBeneficiaries23.csv")
# psfua23.to_csv(psfua23_path, index=False)
# psfua23

In [ ]:
# td_dataset = Dataset(config_file, td_sheet)
# td = pd.read_csv(td_path)
# td = dtype_trans(td, td_dataset)

In [ ]:
# td23 = filter_df_by_date(td, td_dataset.dvars, threshold)
# td23 = filter_df_by_date(td23, td_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# td23_latestdate = td23[td_dataset.dvars].max(axis=1)
# td23.insert(4, "service", "td")
# td23.insert(5, "date", td23_latestdate)
# td23 = td23[['rid', 'fcid', 'service', 'date']]
# td23 = pd.merge(td23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# td23 = pd.merge(td23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# td23_path = os.path.join(root, data, processed, "unvfvt23", "tdBeneficiaries23.csv")
# td23.to_csv(td23_path, index=False)

In [ ]:
# pei_dataset = Dataset(config_file, pei_sheet)
# pei = pd.read_csv(pei_path)
# pei = dtype_trans(pei, pei_dataset)

In [ ]:
# pei23 = filter_df_by_date(pei, pei_dataset.dvars, threshold)
# pei23 = filter_df_by_date(pei23, pei_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# pei23_latestdate = pei23[pei_dataset.dvars].max(axis=1)
# pei23.insert(4, "service", "pei")
# pei23.insert(5, "date", pei23_latestdate)
# pei23 = pei23[['rid', 'fcid', 'service', 'date', "pei_total"]]
# pei23 = pd.merge(pei23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# pei23 = pd.merge(pei23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# pei23_path = os.path.join(root, data, processed, "unvfvt23", "peiBeneficiaries23.xlsx")
# pei23.to_csv(pei23_path, index=False)
# pei23.to_excel(pei23_path, index=False)

In [ ]:
# gpt_dataset = Dataset(config_file, gpt_sheet)
# gpt = pd.read_csv(gpt_path)
# gpt = dtype_trans(gpt, gpt_dataset)
# non_nat_counts = gpt[gpt_dataset.dvars].notna().sum(axis=1)
# gpt['pt_g_total'] = np.where(gpt['pt_g_total'] == 0, non_nat_counts, gpt['pt_g_total'])

In [ ]:
# gpt23 = filter_df_by_date(gpt, gpt_dataset.dvars, threshold)
# gpt23 = filter_df_by_date(gpt23, gpt_dataset.dvars, upper_threshold, comparison_type='b')
# gpt_mask = (gpt23[gpt_dataset.dvars] > threshold) & (gpt23[gpt_dataset.dvars] < upper_threshold)
# gpt23_latestdate = gpt23[gpt_dataset.dvars].where(gpt_mask).max(axis=1)
# gpt23.insert(4, "service", "gpt")
# gpt23.insert(5, "date", gpt23_latestdate)
# gpt23 = gpt23[['rid', 'fcid', 'service', 'date', 'pt_g_total']]

In [ ]:
# gpt23 = pd.merge(gpt23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# gpt23 = pd.merge(gpt23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# gpt23_path = os.path.join(root, data, processed, "unvfvt23", "gptBeneficiaries23.csv")
# gpt23.to_csv(gpt23_path, index=False)

In [ ]:
# ipt_dataset = Dataset(config_file, ipt_sheet)
# ipt = pd.read_csv(ipt_path)
# ipt = dtype_trans(ipt, ipt_dataset)
# non_nat_counts = ipt[ipt_dataset.dvars].notna().sum(axis=1)
# ipt['pt_i_total'] = np.where(ipt['pt_i_total'] == 0, non_nat_counts, ipt['pt_i_total'])

In [ ]:
# ipt23 = filter_df_by_date(ipt, ipt_dataset.dvars, threshold)
# ipt23 = filter_df_by_date(ipt23, ipt_dataset.dvars, threshold=upper_threshold, comparison_type='b')
# ipt_mask = (ipt23[ipt_dataset.dvars] > threshold) & (ipt23[ipt_dataset.dvars] < upper_threshold)
# ipt23_latestdate = ipt23[ipt_dataset.dvars].where(ipt_mask).max(axis=1)
# ipt23.insert(4, "service", "ipt")
# ipt23.insert(5, "date", ipt23_latestdate)
# ipt23 = ipt23[['rid', 'fcid', 'service', 'date', 'pt_i_total']]

In [ ]:
# ipt23 = pd.merge(ipt23, unique_intake[["rid", "family_of_missing", "survivor_of_detention", "primary_torture", "secondary_torture", "sexual_violation_a", "sexual_violation_p", "hrd", "journalist", "wov", "stgbv", "lgbti", "other"]], on='rid', how='left')
# ipt23 = pd.merge(ipt23, unique_scr[['rid', 'sex', 'age']], on='rid', how='left')

In [ ]:
# ipt23_path = os.path.join(root, data, processed, "unvfvt23", "iptBeneficiaries23.csv")
# ipt23.to_csv(ipt23_path, index=False)

In [ ]:
ptint_dataset = Dataset(config_file, ptint_sheet)
ptint = pd.read_csv(ptint_path)
ptint = dtype_trans(ptint, ptint_dataset)

In [ ]:
ptint23 = filter_df_by_date(ptint, ptint_dataset.dvars, threshold)
ptint23 = filter_df_by_date(ptint23, ptint_dataset.dvars, threshold=upper_threshold, comparison_type='b')
ptint23_latestdate = ptint23[['pt_nt_s1', 'pt_nt_s2']].max(axis=1)
_ptint23 = ptint23.copy()
_ptint23.insert(4, 'service', 'ptint')
_ptint23.insert(5, 'date', ptint23_latestdate)
_ptint23 = _ptint23[['rid', 'fcid', 'service', 'date']]
_ptint23

In [ ]:
ptreint23 = filter_df_by_date(ptint23, ['pt_nt_re'], threshold)
ptreint23